In [1]:
import pandas as pd
import glob
import dataset
from datetime import datetime as dt
import sys
import os
import zipfile
from io import BytesIO
from tempfile import NamedTemporaryFile

# Load data

In this step, we load and parse the raw data for each year since 1991, the first year of availability. Unlike the SHR and Return A data, each year of NIBRS data is composed of multiple tables — each table representing a separate concept (e.g., one for victim information, one for incident information, et cetera). The code below loads the tables relevant to our analysis.

In [2]:
BASE_PATH = "../../data/raw/nibrs/"
DB_PATH = "sqlite:///../../data/raw/nibrs-temporary-db.sqlite"

In [3]:
variables = (
    pd.read_csv("../../data/documentation/nibrs-variables.csv")
    .assign(
        variable_name = lambda df: df["variable_name"].str.replace(r"[^A-Z0-9]+", " ").str.strip(),
        segment = lambda df: df["variable_num"].str.slice(0, 2).str.replace(r"V", r"0"),
    )
)    

variables.head()

,year,variable_num,variable_name,location_start,width,segment
0,1991,B1001,SEGMENT LEVEL,1,2,B1
1,1991,B1002,NUMERIC STATE CODE,3,2,B1
2,1991,B1003,ORIGINATING AGENCY IDENTIFIER ORI,5,9,B1
3,1991,B1004,INCIDENT NUMBER,14,12,B1
4,1991,B1005,DATE ORI WAS ADDED,26,8,B1


In [4]:
variables["segment"].value_counts().sort_index()

01     396
02     558
03     624
04    1371
05     264
06     520
07     468
B1     440
B2     484
B3     660
BH     240
W1     655
W3     884
W6     762
Name: segment, dtype: int64

In [5]:
MAIN_OFFENSE_CODES = [
    "09A", # Murder/Nonnegligent Manslaughter
    "09B", # Negligent Manslaughter
    "09C", # Justifiable Homicide
    "11A", # Rape
    "11B", # Sodomy
    "11C", # Sexual Assault With An Object
    "11D", # Fondling (Indecent Liberties/Child Molesting)
    "120", # Robbery
    "13A", # Aggravated Assault
]

In [6]:
segments = variables["variable_num"].str.slice(0, 2).value_counts().sort_index().index.tolist()
segments

['B1',
 'B2',
 'B3',
 'BH',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'W1',
 'W3',
 'W6']

In [7]:
SEGMENTS_TO_PARSE = [
    "B1",
    "B2",
    "B3",
    "BH",
    "01",
    "02",
    "04",
    "06",
]

In [8]:
def load_annual_data_with_bytesio(year, fp):
    streams = dict((seg, NamedTemporaryFile("wb")) for seg in SEGMENTS_TO_PARSE)
    
    for i, line in enumerate(fp):
        if i % 100000 == 0:
            sys.stderr.write(f"\r{i:,}")
            sys.stderr.flush()
        seg = line[:2].decode("utf-8")
        if seg in SEGMENTS_TO_PARSE:
            streams[seg].write(line)
        
    parsed = {}
    
    for seg, stream in streams.items():
        print(seg)
        segvars = (
            variables
            .loc[lambda df: df["year"] == year]
            .loc[lambda df: df["variable_num"].str.slice(0, 2) == seg.replace("0", "V")]
        )
        widths = segvars["width"].tolist()
        names = segvars["variable_name"].tolist()
        stream.seek(0)
        try:
            parsed[seg] = (
                pd.read_fwf(
                    stream.name,
                    widths = widths,
                    names = names,
                    encoding = "latin-1",
                    dtype = str,
                    keep_default_na = False
                )
                .assign(
                    file_year = year                
                )
            )
            
        except pd.errors.EmptyDataError:
            continue
        stream.close()
        
    if "BH" in parsed and len(parsed["BH"]):
        headers = parsed["BH"]
    else:
        headers = parsed["B1"]
        for key in [ "B2", "B3" ]:
            headers = (
                pd.concat([
                    headers,
                    parsed[key]
                    .drop(columns = [
                        "file_year",
                        "SEGMENT LEVEL",
                        "NUMERIC STATE CODE",
                        "ORIGINATING AGENCY IDENTIFIER ORI",
                        "INCIDENT NUMBER",
                        "FILLER BLANKS"                    
                    ])
                ], axis = 1)
            )
            
    return {
        "headers": headers,
        "incidents": parsed["01"],
        "offenses": parsed["02"],
        "victims": parsed["04"],
        "arrestees": parsed["06"]
    }

In [9]:
def make_incident_uid(df):
    # Agency ORI + incident year + incident number
    return df[df.columns[2]] + "|" + df[df.columns[4]].str.slice(0, 4) + "|" + df[df.columns[3]]

In [10]:
def subset_annual_data(data):
    for segment in [ "incidents", "offenses", "victims", "arrestees" ]:
        seg = data[segment]
        seg["incident_uid"] = make_incident_uid(seg)
        seg["incident_submission_uid"] = (
            seg["file_year"].astype(str)
            + "-" 
            + seg["incident_uid"]
        )
            
    offense_incident_uids = data["offenses"][
        data["offenses"]["UCR OFFENSE CODE"].isin(MAIN_OFFENSE_CODES)
    ]["incident_uid"]

    offenses_subset = (
        data["offenses"]
        .loc[lambda df: df["incident_uid"].isin(offense_incident_uids)]
    )

    # All victims, regardless of crime type
    victim_counts = ( 
        data["victims"]
        .groupby("incident_uid")
        .size()
        .to_frame("victim_count")
        .reset_index()
    )    
    
    victims_subset = (
        data["victims"]
        .loc[lambda df: df["incident_uid"].isin(offense_incident_uids)]
    )

    arrestees_subset = (
        data["arrestees"]
        .loc[lambda df: df["incident_uid"].isin(offense_incident_uids)]
    )
        
    incidents_subset = (
        data["incidents"]
        .loc[lambda df: df["incident_uid"].isin(offense_incident_uids)]
        .merge(
            victim_counts,
            how = "left",
            on = [ "incident_uid" ]
        )
    )    
        
    return {
        "headers": data["headers"],
        "incidents": incidents_subset,
        "offenses": offenses_subset,
        "victims": victims_subset,
        "arrestees": arrestees_subset,
    }

In [11]:
%%time
db = dataset.connect(DB_PATH)
for year in range(1991, 2017):
    sys.stderr.write(f"{year}\n")
    sys.stderr.flush()
    
    zip_path = [ x for x in glob.glob(BASE_PATH + "/*.zip")
         if str(year) in x.split("/")[-1] ][0]
                     
    sys.stderr.write(zip_path.split("/")[0] + "\n")
                     
    archive = zipfile.ZipFile(zip_path, "r")
    with archive.open(archive.namelist()[0]) as f:
        parsed = load_annual_data_with_bytesio(year, f)

    sys.stderr.write("\nSubsetting ...\n")
    subsetted = subset_annual_data(parsed)
    del parsed
    sys.stderr.write("Updating tables ...\n")
    for tablename in [ "headers" , "incidents", "offenses", "victims", "arrestees" ]:
        print(tablename, year)
        table = db[tablename]
        if "file_year" in table.columns:
            print("[Clearing previous table]")
            db.query(
                f"DELETE FROM {tablename} WHERE file_year = {year};",
            )
        table.insert_many((row for ix, row in subsetted[tablename].iterrows()))
    del subsetted

1991
..
3,400,000

B1
B2
B3
BH
01
02
04
06



Subsetting ...
Updating tables ...


headers 1991
[Clearing previous table]
incidents 1991
[Clearing previous table]
offenses 1991
[Clearing previous table]
victims 1991
[Clearing previous table]
arrestees 1991
[Clearing previous table]


1992
..
4,500,000

...

headers 2015
[Clearing previous table]
incidents 2015
[Clearing previous table]
offenses 2015
[Clearing previous table]
victims 2015
[Clearing previous table]
arrestees 2015
[Clearing previous table]


2016
..
33,600,000

B1
B2
B3
BH
01
02
04
06



Subsetting ...
Updating tables ...


headers 2016
[Clearing previous table]
incidents 2016
[Clearing previous table]
offenses 2016
[Clearing previous table]
victims 2016
[Clearing previous table]
arrestees 2016
[Clearing previous table]
CPU times: user 3h 19min 12s, sys: 19min 48s, total: 3h 39min 1s
Wall time: 3h 48min 44s


# Save agency metadata

In [12]:
db = dataset.connect(DB_PATH)

In [13]:
agency_metadata = (
    pd.read_sql_table("headers", db.executable)
    .rename(columns = {
        "MASTER FILE YEAR": "year",
        "ORIGINATING AGENCY IDENTIFIER ORI": "agency_ori_9",
        "POPULATION GROUP": "ori_group",
        "CURRENT POPULATION 1": "pop1",
        "CURRENT POPULATION 2": "pop2",
        "CURRENT POPULATION 3": "pop3",
        "CURRENT POPULATION 4": "pop4",
        "CURRENT POPULATION 5": "pop5",
        "NUMBER OF MONTHS REPORTED": "num_months",
    })
    [[
        "agency_ori_9",
        "year",
        "ori_group",
        "pop1",
        "pop2",
        "pop3",
        "pop4",
        "pop5",
        "num_months",
    ]]
    .drop_duplicates()
    .astype({
        "pop1": int,
        "pop2": int,
        "pop3": int,
        "pop4": int,
        "pop5": int,
        "num_months": int,
    })
    .sort_values([
        "agency_ori_9",
        "year"
    ])
)

agency_metadata.head()

,agency_ori_9,year,ori_group,pop1,pop2,pop3,pop4,pop5,num_months
17962,AK0010100,1991,2,234571,0,0,0,0,0
36172,AK0010100,1992,2,241565,0,0,0,0,0
54544,AK0010100,1993,1C,250720,0,0,0,0,0
73030,AK0010100,1994,1C,253667,0,0,0,0,0
91677,AK0010100,1995,1C,253500,0,0,0,0,0


In [14]:
agency_metadata.to_csv(
    "../../data/standardized/nibrs-agency-metadata.csv",
    index = False
)

# Deduplicate reports

## Incidents

In [15]:
incident_submissions = pd.read_sql_table("incidents", db.executable)

In [16]:
(
    incident_submissions
    ["incident_uid"]
    .value_counts()
    .value_counts()
)

1    5403443
2     112043
3          1
Name: incident_uid, dtype: int64

In [17]:
(
    incident_submissions
    .groupby([
        "incident_uid",
        "INCIDENT DATE"
    ])
    .size()
    .value_counts()
)

1    5403443
2     112043
3          1
dtype: int64

In [18]:
(
    incident_submissions
    .assign(year = lambda df: df["INCIDENT DATE"].str.slice(0, 4))
    .groupby([
        "incident_uid",
        "year"
    ])
    .size()
    .to_frame("num")
    .reset_index()
    .groupby([ "year", "num" ])
    .size()
    .unstack()
    .fillna("")
#     .value_counts()
)

num,1,2,3
year,,,
1991,60714.0,741,
1992,64954.0,1152,
1993,65844.0,1138,
1994,64837.0,1294,
1995,58251.0,1040,
1996,71970.0,1554,
1997,92678.0,2061,
1998,113467.0,2446,
1999,129618.0,2483,


In [19]:
(
    incident_submissions
    ["incident_uid"]
    .value_counts()
    .head()
)

KYKSP0800|2011|294 X2YC-F39    3
KYKSP0700|2010|62136V0MPU72    2
MI8008000|2015|1H0UQU72J404    2
KY0790000|2010|113PQHFU6X I    2
IA0340100|1992|66- 0MP 728N    2
Name: incident_uid, dtype: int64

In [20]:
(
    incident_submissions
    ["incident_uid"]
    .value_counts()
    .head()
)

KYKSP0800|2011|294 X2YC-F39    3
KYKSP0700|2010|62136V0MPU72    2
MI8008000|2015|1H0UQU72J404    2
KY0790000|2010|113PQHFU6X I    2
IA0340100|1992|66- 0MP 728N    2
Name: incident_uid, dtype: int64

In [21]:
(
    incident_submissions
    .loc[lambda df: df["incident_uid"] == "MI4183400|2001|1I00Q-H39L1H"]
    .T
)

,973899,1175838
id,1237605,1439544
SEGMENT LEVEL,01,01
NUMERIC STATE CODE,21,21
ORIGINATING AGENCY IDENTIFIER ORI,MI4183400,MI4183400
INCIDENT NUMBER,1I00Q-H39L1H,1I00Q-H39L1H
INCIDENT DATE,20011102,20011102
REPORT DATE INDICATOR,,
INCIDENT DATE HOUR,00,00
TOTAL OFFENSE SEGMENTS,01,01
TOTAL VICTIM SEGMENTS,001,001


In [22]:
incidents = (
    incident_submissions
    .sort_values("file_year", ascending = False)
    .drop_duplicates(subset = [ "incident_uid" ])
    .replace({
        "EXCEPTIONAL CLEARANCE DATE": {
            "": pd.np.nan
        }
    })
    .assign(
        occurred_date = lambda df: df["INCIDENT DATE"],
        year = lambda df: df["INCIDENT DATE"].str.slice(0, 4).astype(int),
        month = lambda df: df["INCIDENT DATE"].str.slice(4, 6).astype(int),
        arrest_ind = lambda df: (df["TOTAL ARRESTEE SEGMENTS"].astype(int) > 0).astype(int),
        exc_clearance_ind = lambda df: df["EXCEPTIONAL CLEARANCE DATE"].notnull().astype(int),
        exc_clearance_date = lambda df: df["EXCEPTIONAL CLEARANCE DATE"],
    )
    .assign(
        clearance_ind = lambda df: (df["arrest_ind"] | df["exc_clearance_ind"]).astype(int),    
    )
)

incidents.head()

,id,SEGMENT LEVEL,NUMERIC STATE CODE,ORIGINATING AGENCY IDENTIFIER ORI,INCIDENT NUMBER,INCIDENT DATE,REPORT DATE INDICATOR,INCIDENT DATE HOUR,TOTAL OFFENSE SEGMENTS,TOTAL VICTIM SEGMENTS,...,incident_submission_uid,victim_count,CARGO THEFT,occurred_date,year,month,arrest_ind,exc_clearance_ind,exc_clearance_date,clearance_ind
5627531,5891237,01,48,WI0700000,F7W-J8N4P-F3,20160822,,06,01,001,...,2016-WI0700000|2016|F7W-J8N4P-F3,1,,20160822,2016,8,0,0,NaN,0
5389783,5653489,01,21,MI5071300,W10PM6X CX4E,20160405,,22,01,001,...,2016-MI5071300|2016|W10PM6X CX4E,1,,20160405,2016,4,0,0,NaN,0
5389775,5653481,01,21,MI5071300,9C0W1OZLQV83,20160326,,02,01,001,...,2016-MI5071300|2016|9C0W1OZLQV83,1,,20160326,2016,3,0,0,NaN,0
5389776,5653482,01,21,MI5071300,W10PM6X 314I,20160331,,08,01,001,...,2016-MI5071300|2016|W10PM6X 314I,1,,20160331,2016,3,0,0,NaN,0
5389777,5653483,01,21,MI5071300,FV0GT4KOEV7Z,20160306,,02,01,001,...,2016-MI5071300|2016|FV0GT4KOEV7Z,1,,20160306,2016,3,1,0,NaN,1


In [23]:
assert incidents["incident_uid"].value_counts().max() == 1
del incident_submissions

## Offenses

In [24]:
offenses = (
    pd.read_sql_table("offenses", db.executable)
    .assign(
        offense_uid = lambda df: df["incident_uid"] + "-" + df["UCR OFFENSE CODE"]
    )
    .loc[lambda x: x["incident_submission_uid"].isin(incidents["incident_submission_uid"])]
)

offenses.head()

,id,SEGMENT LEVEL,NUMERIC STATE CODE,ORIGINATING AGENCY IDENTIFIER ORI,INCIDENT NUMBER,INCIDENT DATE,UCR OFFENSE CODE,OFFENSE ATTEMPTED COMPLETED,OFFENDER S SUSPECTED OF USING 1,OFFENDER S OF USING 2,...,N 02 RECORDS PER ORI INCIDENT NUMBER,file_year,incident_uid,incident_submission_uid,BIAS MOTIVATION 1,BIAS MOTIVATION 2,BIAS MOTIVATION 3,BIAS MOTIVATION 4,BIAS MOTIVATION 5,offense_uid
0,225251,02,01,AL0010000,6PMRWOZOJMQ-,19910416,11A,C,N,,...,,1991,AL0010000|1991|6PMRWOZOJMQ-,1991-AL0010000|1991|6PMRWOZOJMQ-,None,None,None,None,None,AL0010000|1991|6PMRWOZOJMQ--11A
1,225252,02,01,AL0010000,6PM4KOKTTMQ-,19910112,13A,C,N,,...,,1991,AL0010000|1991|6PM4KOKTTMQ-,1991-AL0010000|1991|6PM4KOKTTMQ-,None,None,None,None,None,AL0010000|1991|6PM4KOKTTMQ--13A
2,225253,02,01,AL0010000,6PMU728PUMQ-,19910215,13A,C,N,,...,,1991,AL0010000|1991|6PMU728PUMQ-,1991-AL0010000|1991|6PMU728PUMQ-,None,None,None,None,None,AL0010000|1991|6PMU728PUMQ--13A
3,225254,02,01,AL0010000,UKMIYQ4F W5H,19910228,13A,C,N,,...,,1991,AL0010000|1991|UKMIYQ4F W5H,1991-AL0010000|1991|UKMIYQ4F W5H,None,None,None,None,None,AL0010000|1991|UKMIYQ4F W5H-13A
4,225255,02,01,AL0010000,3XP04SOSPMQ-,19910829,13A,C,N,,...,,1991,AL0010000|1991|3XP04SOSPMQ-,1991-AL0010000|1991|3XP04SOSPMQ-,None,None,None,None,None,AL0010000|1991|3XP04SOSPMQ--13A


In [25]:
x = (
    offenses
    .groupby([
        "incident_uid",
        "UCR OFFENSE CODE"
    ])
    .size()
    .loc[lambda x: x > 1]
)

In [26]:
assert (
    offenses
    .groupby([
        "incident_uid",
        "UCR OFFENSE CODE"
    ])
    .size()
    .max() == 1
)

In [27]:
(
    offenses
    ["UCR OFFENSE CODE"]
    .value_counts(normalize = True)
    .sort_index()
    .round(4)
)

09A    0.0089
09B    0.0006
09C    0.0003
100    0.0117
11A    0.0745
11B    0.0176
11C    0.0085
11D    0.0805
120    0.1960
13A    0.4858
13B    0.0205
13C    0.0031
200    0.0006
210    0.0001
220    0.0136
23A    0.0001
23B    0.0001
23C    0.0019
23D    0.0012
23E    0.0000
23F    0.0008
23G    0.0002
23H    0.0043
240    0.0023
250    0.0003
26A    0.0006
26B    0.0002
26C    0.0007
26D    0.0000
26E    0.0000
26F    0.0000
26G    0.0000
270    0.0000
280    0.0011
290    0.0305
35A    0.0084
35B    0.0033
36A    0.0001
36B    0.0002
370    0.0006
39A    0.0000
39B    0.0000
39C    0.0000
40A    0.0001
40B    0.0001
40C    0.0000
510    0.0000
520    0.0204
64A    0.0000
64B    0.0000
720    0.0000
Name: UCR OFFENSE CODE, dtype: float64

### Add weapon information

In [28]:
(
    variables
    .loc[lambda df: df["variable_name"] == "TYPE WEAPON FORCE INVOLVED 1"]
    .drop_duplicates(subset = [ "variable_num" ])
)

,year,variable_num,variable_name,location_start,width,segment
103,1991,V2017,TYPE WEAPON FORCE INVOLVED 1,49,3,02


In [29]:
translations = pd.read_csv(
    "../../data/documentation/nibrs-translations.csv"
)
translations.head()

,variable_num,value,translation
0,BH001,BH,Batch Header
1,BH002,01,AL-Alabama
2,BH002,02,AZ-Arizona
3,BH002,03,AR-Arkansas
4,BH002,04,CA-California


In [30]:
weapon_translations = (
    translations
    .loc[lambda df: df["variable_num"] == "V2017"]
    .set_index("value")
    ["translation"]
    .to_dict()
)

weapon_translations

{'11': 'Firearm (type not stated)',
 '11A': 'Firearm (type not stated), automatic',
 '12': 'Handgun',
 '12A': 'Handgun, automatic',
 '13': 'Rifle',
 '13A': 'Rifle, automatic',
 '14': 'Shotgun',
 '14A': 'Shotgun, automatic',
 '15': 'Other Firearm',
 '15A': 'Other Firearm, automatic',
 '20': 'Knife/Cutting Instrument (ice pick, screwdriver, ax, etc.)',
 '30': 'Blunt Object (club, hammer, etc.)',
 '35': 'Motor Vehicle',
 '40': 'Personal Weapons (hands, feet, teeth, etc.)',
 '50': 'Poison (include gas as in code 85 asphyxiation)',
 '60': 'Explosives',
 '65': 'Fire/Incendiary Device',
 '70': 'Drugs/Narcotics/Sleeping Pills',
 '85': 'Asphyxiation (by drowning, strangulation, suffocation, gas, etc.)',
 '90': 'Other',
 '95': 'Unknown',
 '99': 'None'}

In [31]:
offense_weapons = (
    offenses
    [[ "offense_uid" ] + [ c for c in offenses.columns if "TYPE WEAPON" in c]]
    .melt(id_vars = [ "offense_uid" ])
    .loc[lambda df: ~df["value"].isin([
        "",
        pd.np.nan
    ])]
)

offense_weapons.head()

,offense_uid,variable,value
0,AL0010000|1991|6PMRWOZOJMQ--11A,TYPE WEAPON FORCE INVOLVED 1,90
1,AL0010000|1991|6PM4KOKTTMQ--13A,TYPE WEAPON FORCE INVOLVED 1,20
2,AL0010000|1991|6PMU728PUMQ--13A,TYPE WEAPON FORCE INVOLVED 1,90
3,AL0010000|1991|UKMIYQ4F W5H-13A,TYPE WEAPON FORCE INVOLVED 1,14
4,AL0010000|1991|3XP04SOSPMQ--13A,TYPE WEAPON FORCE INVOLVED 1,90


In [32]:
(
    offense_weapons
    .groupby("value")
    .size()
    .to_frame("count")
    .reset_index()
    .assign(
        weapon = lambda df: df["value"].apply(weapon_translations.get)
    )
)

,value,count,weapon
0,11,236132,Firearm (type not stated)
1,11A,14277,"Firearm (type not stated), automatic"
2,12,875550,Handgun
3,12A,76824,"Handgun, automatic"
4,13,42925,Rifle
5,13A,4036,"Rifle, automatic"
6,14,74971,Shotgun
7,14A,1518,"Shotgun, automatic"
8,15,37612,Other Firearm
9,15A,1576,"Other Firearm, automatic"


In [33]:
WEAPON_CATEGORIES = {
    "11": "firearm",
    "11A": "firearm",
    "12": "firearm",
    "12A": "firearm",
    "13": "firearm",
    "13A": "firearm",
    "14": "firearm",
    "14A": "firearm",
    "15": "firearm",
    "15A": "firearm",
    "20": "knife",
    "30": "contact",
    "35": "other",
    "40": "contact",
    "50": "other",
    "60": "other",
    "65": "other",
    "70": "other",
    "85": "contact",
    "90": "other",
    "95": "missing",
    "99": "missing",
}

In [34]:
FIREARM_WEAPON_CODES = [
    "11",  # Firearm (type not stated)
    "11A", # Firearm (type not stated), automatic
    "12",  # Handgun
    "12A", # Handgun, automatic
    "13",  # Rifle
    "13A", # Rifle, automatic
    "14",  # Shotgun
    "14A", # Shotgun, automatic
    "15",  # Other Firearm
    "15A", # Other Firearm, automatic
]

In [35]:
offense_primary_weapons = (
    offense_weapons
    .loc[lambda x: x["variable"] == "TYPE WEAPON FORCE INVOLVED 1"]
    .assign(
        primary_weapon_category = lambda df: df["value"].apply(WEAPON_CATEGORIES.get)
    )
    [[
        "offense_uid",
        "primary_weapon_category"
    ]]
    .merge(
        offenses[[ "offense_uid", "incident_uid", "UCR OFFENSE CODE" ]],
        how = "left",
        on = "offense_uid"
    )
)

offense_primary_weapons.head()

,offense_uid,primary_weapon_category,incident_uid,UCR OFFENSE CODE
0,AL0010000|1991|6PMRWOZOJMQ--11A,other,AL0010000|1991|6PMRWOZOJMQ-,11A
1,AL0010000|1991|6PM4KOKTTMQ--13A,knife,AL0010000|1991|6PM4KOKTTMQ-,13A
2,AL0010000|1991|6PMU728PUMQ--13A,other,AL0010000|1991|6PMU728PUMQ-,13A
3,AL0010000|1991|UKMIYQ4F W5H-13A,firearm,AL0010000|1991|UKMIYQ4F W5H,13A
4,AL0010000|1991|3XP04SOSPMQ--13A,other,AL0010000|1991|3XP04SOSPMQ-,13A


In [36]:
offenses = (
    offenses
    .drop(columns = [ "primary_weapon_category" ], errors = "ignore")
    .merge(
        offense_primary_weapons,
        on = [ "offense_uid", "incident_uid", "UCR OFFENSE CODE" ],
        how = "left"
    )
)

offenses["primary_weapon_category"].value_counts(normalize = True)

contact    0.423284
firearm    0.226040
knife      0.128292
missing    0.122118
other      0.100266
Name: primary_weapon_category, dtype: float64

In [37]:
offenses["firearm_ind"] = offenses["offense_uid"].isin(
    offense_weapons[
        offense_weapons["value"].isin(FIREARM_WEAPON_CODES)
    ]["offense_uid"]
).astype(int)

offenses["firearm_ind"].value_counts(normalize = True)

0    0.78824
1    0.21176
Name: firearm_ind, dtype: float64

## Arrests

In [38]:
arrestees = (
    pd.read_sql_table("arrestees", db.executable)
    .loc[lambda x: x["incident_submission_uid"].isin(incidents["incident_submission_uid"])]
)

arrestees.head()

,id,SEGMENT LEVEL,NUMERIC STATE CODE,ORIGINATING AGENCY IDENTIFIER ORI,INCIDENT NUMBER,INCIDENT DATE,ARRESTEE SEQUENCE NUMBER,ARREST TRANSACTION NUMBER,ARREST DATE,TYPE OF ARREST,...,SEX OF ARRESTEE,RACE OF ARRESTEE,ETHNICITY OF ARRESTEE,RESIDENT STATUS OF ARRESTEE,DISPOSITION OF ARRESTEE UNDER 18,N 06 RECORDS PER ORI INCIDENT NUMBER,file_year,incident_uid,incident_submission_uid,ARRESTEE SEGMENT LEVEL
0,47849,06,01,AL0010000,S7PD0AKZMYBR,19910802,01,S7PD0AKZMYBR,19910802,O,...,M,W,U,R,,,1991,AL0010000|1991|S7PD0AKZMYBR,1991-AL0010000|1991|S7PD0AKZMYBR,None
1,47850,06,01,AL0010000,IWPRUA83TL6X,19910714,01,IWPRUA83TL6X,19910715,T,...,M,W,U,R,,,1991,AL0010000|1991|IWPRUA83TL6X,1991-AL0010000|1991|IWPRUA83TL6X,None
2,47851,06,01,AL0010000,-LP9QRJV5AEM,19910526,01,-LP9QRJV5AEM,19910531,T,...,F,W,U,R,,,1991,AL0010000|1991|-LP9QRJV5AEM,1991-AL0010000|1991|-LP9QRJV5AEM,None
3,47852,06,01,AL0010000,-LP9QUVV5AEM,19910515,01,-LP9QUVV5AEM,19910515,O,...,M,W,U,R,,,1991,AL0010000|1991|-LP9QUVV5AEM,1991-AL0010000|1991|-LP9QUVV5AEM,None
4,47853,06,01,AL0010000,-LP9QUVV5AEM,19910515,02,-LP9QUVV5AEM,19910516,O,...,M,B,U,R,R,,1991,AL0010000|1991|-LP9QUVV5AEM,1991-AL0010000|1991|-LP9QUVV5AEM,None


In [39]:
arrestees.columns

Index(['id', 'SEGMENT LEVEL', 'NUMERIC STATE CODE',
       'ORIGINATING AGENCY IDENTIFIER ORI', 'INCIDENT NUMBER', 'INCIDENT DATE',
       'ARRESTEE SEQUENCE NUMBER', 'ARREST TRANSACTION NUMBER', 'ARREST DATE',
       'TYPE OF ARREST', 'MULTIPLE ARRESTEE SEGMENTS INDICATOR',
       'UCR ARREST OFFENSE CODE', 'ARRESTEE ARMED WITH 1',
       'ARRESTEE ARMED WITH 2', 'AGE OF ARRESTEE', 'SEX OF ARRESTEE',
       'RACE OF ARRESTEE', 'ETHNICITY OF ARRESTEE',
       'RESIDENT STATUS OF ARRESTEE', 'DISPOSITION OF ARRESTEE UNDER 18',
       'N 06 RECORDS PER ORI INCIDENT NUMBER', 'file_year', 'incident_uid',
       'incident_submission_uid', 'ARRESTEE SEGMENT LEVEL'],
      dtype='object')

In [40]:
first_arrest_dates = (
    arrestees
    .groupby([
        "incident_submission_uid",
    ])
    ["ARREST DATE"]
    .min()
    .to_frame("arrest_date")
    .reset_index()
)

first_arrest_dates.head()

,incident_submission_uid,arrest_date
0,1991-AL0010000|1991|-LP9QRJV5AEM,19910531
1,1991-AL0010000|1991|-LP9QUVV5AEM,19910515
2,1991-AL0010000|1991|-LPNGBRK5AEM,19911023
3,1991-AL0010000|1991|-LPNJU435AEM,19911120
4,1991-AL0010000|1991|2DPJQKVCZHPU,19910325


## Victims

In [41]:
victims = (
    pd.read_sql_table("victims", db.executable)
    .assign(
        victim_uid = lambda df: df["incident_uid"] + "-" + df["VICTIM SEQUENCE NUMBER"]
    )
    .loc[lambda x: x["incident_submission_uid"].isin(incidents["incident_submission_uid"])]
)

victims.head()

,id,SEGMENT LEVEL,NUMERIC STATE CODE,ORIGINATING AGENCY IDENTIFIER ORI,INCIDENT NUMBER,INCIDENT DATE,VICTIM SEQUENCE NUMBER,UCR OFFENSE CODE 1,UCR OFFENSE CODE 2,UCR OFFENSE CODE 3,...,OFFENDER NUMBER TO BE RELATED 10TH,RELATIONSHIP OF VICTIM TO OFFENDER 10TH,N 04 RECORDS PER ORI INCIDENT NUMBER,file_year,incident_uid,incident_submission_uid,TYPE OF ACTIVITY OFFICER,ASSIGNMENT TYPE OFFICER,ORI OTHER JURISDICTION OFFICER,victim_uid
0,227009,04,01,AL0010000,6PMRWOZOJMQ-,19910416,001,11A,,,...,,,,1991,AL0010000|1991|6PMRWOZOJMQ-,1991-AL0010000|1991|6PMRWOZOJMQ-,None,None,None,AL0010000|1991|6PMRWOZOJMQ--001
1,227010,04,01,AL0010000,6PM4KOKTTMQ-,19910112,001,13A,,,...,,,,1991,AL0010000|1991|6PM4KOKTTMQ-,1991-AL0010000|1991|6PM4KOKTTMQ-,None,None,None,AL0010000|1991|6PM4KOKTTMQ--001
2,227011,04,01,AL0010000,6PMU728PUMQ-,19910215,001,13A,,,...,,,,1991,AL0010000|1991|6PMU728PUMQ-,1991-AL0010000|1991|6PMU728PUMQ-,None,None,None,AL0010000|1991|6PMU728PUMQ--001
3,227012,04,01,AL0010000,UKMIYQ4F W5H,19910228,001,13A,,,...,,,,1991,AL0010000|1991|UKMIYQ4F W5H,1991-AL0010000|1991|UKMIYQ4F W5H,None,None,None,AL0010000|1991|UKMIYQ4F W5H-001
4,227013,04,01,AL0010000,3XP04SOSPMQ-,19910829,001,13A,,,...,,,,1991,AL0010000|1991|3XP04SOSPMQ-,1991-AL0010000|1991|3XP04SOSPMQ-,None,None,None,AL0010000|1991|3XP04SOSPMQ--001


### Add injury information

In [42]:
victim_injuries = (
    victims
    [[ "victim_uid" ] + [ c for c in victims.columns if "TYPE INJURY" in c]]
    .pipe(pd.melt, id_vars = [ "victim_uid" ])
    .loc[lambda df: (
        (df["variable"] == "TYPE INJURY 1") |
        (df["value"].notnull() & (df["value"] != ""))
    )]
)

victim_injuries.head()

,victim_uid,variable,value
0,AL0010000|1991|6PMRWOZOJMQ--001,TYPE INJURY 1,M
1,AL0010000|1991|6PM4KOKTTMQ--001,TYPE INJURY 1,M
2,AL0010000|1991|6PMU728PUMQ--001,TYPE INJURY 1,N
3,AL0010000|1991|UKMIYQ4F W5H-001,TYPE INJURY 1,N
4,AL0010000|1991|3XP04SOSPMQ--001,TYPE INJURY 1,M


In [43]:
(
    victim_injuries
    .fillna("[missing]")
    .groupby("variable")
    ["value"].value_counts(normalize = True)
    .unstack().T
    .sort_values("TYPE INJURY 1", ascending = False)
)

variable,TYPE INJURY 1,TYPE INJURY 2,TYPE INJURY 3,TYPE INJURY 4,TYPE INJURY 5
value,,,,,
N,0.522057,NaN,NaN,NaN,NaN
M,0.241852,0.225320,0.123859,0.068508,0.053061
,0.093824,NaN,NaN,NaN,NaN
L,0.055342,0.205061,0.160170,0.104176,0.065306
O,0.038963,0.187474,0.264960,0.304263,0.259864
I,0.019720,0.156710,0.104579,0.063941,0.039456
B,0.018587,0.050605,0.050569,0.054371,0.048980
U,0.006370,0.125431,0.237990,0.339060,0.458503
T,0.003285,0.049399,0.057873,0.065681,0.074830


In [44]:
MAJOR_INJURY_CODES = [
    "L", # Severe Laceration
    "O", # Other Major Injury
    "B", # Apparent Broken Bones
    "I", # Possible Internal Injury
    "U", # Unconsciousness
    "T", # Loss of Teeth
]

MINOR_INJURY_CODES = [ "M" ] # Apparent Minor Injury

INJURY_CODES = MAJOR_INJURY_CODES + MINOR_INJURY_CODES

In [45]:
victims["major_injury_ind"] = victims["victim_uid"].isin(
    victim_injuries[
        victim_injuries["value"].isin(MAJOR_INJURY_CODES)
    ]["victim_uid"]
).astype(int)

In [46]:
# `minor_injury_ind` excludes victims with major injuries

victims["minor_injury_ind"] = (
    victims["victim_uid"].isin(
        victim_injuries[
            victim_injuries["value"].isin(MINOR_INJURY_CODES)
        ]["victim_uid"]
    ) & ~victims["major_injury_ind"]
).astype(int)

In [47]:
victims["injury_data_missing"] = victims["victim_uid"].isin(
    victim_injuries
    .loc[lambda df: df["value"] == ""]
    ["victim_uid"]
)

victims["injury_data_missing"].mean()

0.09382443634672259

In [48]:
victims["victim_injury_category"] = (
    victims
    .pipe(lambda df: pd.np.where(
        df["major_injury_ind"],
        "major",
        pd.np.where(
            df["minor_injury_ind"],
            "minor",
            pd.np.where(
                df["injury_data_missing"],
                "missing",
                "none"
            )
        )
    ))
)

victims["victim_injury_category"].value_counts(normalize = True)

none       0.522057
minor      0.237995
major      0.146124
missing    0.093824
Name: victim_injury_category, dtype: float64

### Classify victim offenses

In [49]:
VICTIM_OFFENSE_COLS = [ c for c in victims.columns
    if "UCR OFFENSE CODE" in c ]
VICTIM_OFFENSE_COLS

['UCR OFFENSE CODE 1',
 'UCR OFFENSE CODE 2',
 'UCR OFFENSE CODE 3',
 'UCR OFFENSE CODE 4',
 'UCR OFFENSE CODE 5',
 'UCR OFFENSE CODE 6',
 'UCR OFFENSE CODE 7',
 'UCR OFFENSE CODE 8',
 'UCR OFFENSE CODE 9',
 'UCR OFFENSE CODE 10']

In [50]:
victim_offenses = (
    victims
    [[ "incident_uid", "victim_uid" ] + VICTIM_OFFENSE_COLS ]
    .melt(id_vars = [ "incident_uid", "victim_uid" ])
    .loc[lambda df: ~df["value"].isin([
        "",
        pd.np.nan
    ])]
    .rename(columns = {
        "value": "UCR OFFENSE CODE"
    })
    .assign(
        offense_uid = lambda df: df["incident_uid"] + "-" + df["UCR OFFENSE CODE"]
    )
)

victim_offenses.head()

,incident_uid,victim_uid,variable,UCR OFFENSE CODE,offense_uid
0,AL0010000|1991|6PMRWOZOJMQ-,AL0010000|1991|6PMRWOZOJMQ--001,UCR OFFENSE CODE 1,11A,AL0010000|1991|6PMRWOZOJMQ--11A
1,AL0010000|1991|6PM4KOKTTMQ-,AL0010000|1991|6PM4KOKTTMQ--001,UCR OFFENSE CODE 1,13A,AL0010000|1991|6PM4KOKTTMQ--13A
2,AL0010000|1991|6PMU728PUMQ-,AL0010000|1991|6PMU728PUMQ--001,UCR OFFENSE CODE 1,13A,AL0010000|1991|6PMU728PUMQ--13A
3,AL0010000|1991|UKMIYQ4F W5H,AL0010000|1991|UKMIYQ4F W5H-001,UCR OFFENSE CODE 1,13A,AL0010000|1991|UKMIYQ4F W5H-13A
4,AL0010000|1991|3XP04SOSPMQ-,AL0010000|1991|3XP04SOSPMQ--001,UCR OFFENSE CODE 1,13A,AL0010000|1991|3XP04SOSPMQ--13A


In [51]:
# Confirming that there are no incident-victim-offense duplicates
assert (
    victim_offenses
    .groupby([
        "victim_uid",
        "UCR OFFENSE CODE"
    ])
    .size()
    .max()
) == 1

In [52]:
victim_top_offenses = (
    victim_offenses
    .loc[lambda df: df["UCR OFFENSE CODE"].isin(MAIN_OFFENSE_CODES)]
    .sort_values("UCR OFFENSE CODE")
    .drop_duplicates(subset = [ "victim_uid" ])
)

print(f"{len(victim_top_offenses):,} incidents")

(
    victim_top_offenses
    ["UCR OFFENSE CODE"]
    .value_counts()
    .sort_index()
)

6,628,142 incidents


09A      60691
09B       3849
09C       2170
11A     489342
11B     116372
11C      55879
11D     574608
120    1627291
13A    3697940
Name: UCR OFFENSE CODE, dtype: int64

In [53]:
victim_top_offenses.head()

,incident_uid,victim_uid,variable,UCR OFFENSE CODE,offense_uid
36153377,VA0370000|1996|Z63BRW5HPU72,VA0370000|1996|Z63BRW5HPU72-001,UCR OFFENSE CODE 6,09A,VA0370000|1996|Z63BRW5HPU72-09A
6610607,VA1230000|2015|94-4NLE8O39G,VA1230000|2015|94-4NLE8O39G-001,UCR OFFENSE CODE 1,09A,VA1230000|2015|94-4NLE8O39G-09A
4408829,OHCOP0000|2010|8I0VYC5VT2 N,OHCOP0000|2010|8I0VYC5VT2 N-001,UCR OFFENSE CODE 1,09A,OHCOP0000|2010|8I0VYC5VT2 N-09A
5042711,CT0004300|2012|2W1J0U73LNK,CT0004300|2012|2W1J0U73LNK-001,UCR OFFENSE CODE 1,09A,CT0004300|2012|2W1J0U73LNK-09A
6610653,VA1230000|2015|7I-IMIR6ZU72,VA1230000|2015|7I-IMIR6ZU72-001,UCR OFFENSE CODE 1,09A,VA1230000|2015|7I-IMIR6ZU72-09A


In [54]:
victim_circumstances = (
    victims
    [[ "victim_uid" ] + [ c for c in victims.columns if "AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES" in c]]
    .pipe(pd.melt, id_vars = [ "victim_uid" ])
    .loc[lambda df: (
        (df["variable"] == "AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 1") |
        (df["value"].notnull() & (df["value"] != ""))
    )]
)

victim_circumstances.head()

,victim_uid,variable,value
0,AL0010000|1991|6PMRWOZOJMQ--001,AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 1,
1,AL0010000|1991|6PM4KOKTTMQ--001,AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 1,10
2,AL0010000|1991|6PMU728PUMQ--001,AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 1,10
3,AL0010000|1991|UKMIYQ4F W5H-001,AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 1,10
4,AL0010000|1991|3XP04SOSPMQ--001,AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 1,10


In [55]:
(
    victim_circumstances
    .loc[lambda df: df["value"] == "RU"]
    .assign(
        year = lambda df: df["victim_uid"].str.slice(10, 14)
    )
    ["year"]
    .value_counts()
    .sort_index()
)

Series([], Name: year, dtype: int64)

In [56]:
victim_circumstances.groupby([ "value", "variable",  ]).size().unstack()

variable,AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 1,AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 2
value,,
,3369677.0,NaN
01,1849987.0,8014.0
02,102423.0,577.0
03,18748.0,1233.0
04,18460.0,676.0
05,13180.0,712.0
06,175672.0,9727.0
07,147.0,2.0
08,36635.0,2909.0


In [57]:
(
    victim_circumstances
    ["value"]
    .replace("", 0)
    .astype(int)
    .value_counts(normalize = True)
    .to_frame("count")
    .sort_index()
)

,count
0,0.469501
1,0.258877
2,0.014351
3,0.002784
4,0.002666
5,0.001936
6,0.025832
7,0.000021
8,0.005510
9,0.099939


In [58]:
victim_relationships = (
    victims
    [[ "victim_uid" ] + [ c for c in victims.columns if "RELATIONSHIP OF VICTIM TO OFFENDER" in c]]
    .pipe(pd.melt, id_vars = [ "victim_uid" ])
    .loc[lambda df: (
        (df["variable"] == "RELATIONSHIP OF VICTIM TO OFFENDER (1ST)") |
        (df["value"].notnull() & (df["value"] != ""))
    )]
)

victim_relationships.head()

,victim_uid,variable,value
0,AL0010000|1991|6PMRWOZOJMQ--001,RELATIONSHIP OF VICTIM TO OFFENDER 1ST,AQ
1,AL0010000|1991|6PM4KOKTTMQ--001,RELATIONSHIP OF VICTIM TO OFFENDER 1ST,IL
2,AL0010000|1991|6PMU728PUMQ--001,RELATIONSHIP OF VICTIM TO OFFENDER 1ST,SE
3,AL0010000|1991|UKMIYQ4F W5H-001,RELATIONSHIP OF VICTIM TO OFFENDER 1ST,PA
4,AL0010000|1991|3XP04SOSPMQ--001,RELATIONSHIP OF VICTIM TO OFFENDER 1ST,AQ


In [59]:
victim_relationships["variable"].value_counts()

RELATIONSHIP OF VICTIM TO OFFENDER 1ST     6067746
RELATIONSHIP OF VICTIM TO OFFENDER 2ND     1465415
RELATIONSHIP OF VICTIM TO OFFENDER 3RD      466548
RELATIONSHIP OF VICTIM TO OFFENDER 4TH      185606
RELATIONSHIP OF VICTIM TO OFFENDER 5TH       75508
RELATIONSHIP OF VICTIM TO OFFENDER 6TH       34115
RELATIONSHIP OF VICTIM TO OFFENDER 7TH       15963
RELATIONSHIP OF VICTIM TO OFFENDER 8TH        8848
RELATIONSHIP OF VICTIM TO OFFENDER 9TH        4823
RELATIONSHIP OF VICTIM TO OFFENDER 10TH       2627
Name: variable, dtype: int64

In [60]:
(
    victim_relationships
    ["value"]
    .value_counts(normalize = True)
    .to_frame("count")
)

,count
ST,0.230477
RU,0.218780
AQ,0.189027
OK,0.076671
BG,0.067255
SE,0.031200
FR,0.028396
CH,0.025642
OF,0.025269
VO,0.023043


In [61]:
REL_DV = [
    "SE", # Victim was Spouse
    "CS", # Victim was Common-Law Spouse
    "BG", # Victim was Boyfriend/Girlfriend
    "HR", # Homosexual Relationship
    "XS", # Victim was Ex-Spouse
]

REL_FV = [
    "SE", # Victim was Spouse
    "CS", # Victim was Common-Law Spouse
    "PA", # Victim was Parent
    "SB", # Victim was Sibling
    "CH", # Victim was Child
    "GP", # Victim was Grandparent
    "GC", # Victim was Grandchild
    "IL", # Victim was In-Law
    "SP", # Victim was Stepparent
    "SC", # Victim was Stepchild
    "SS", # Victim was Stepsibling
    "OF", # Victim was Other Family Member
]

In [62]:
victims["domestic_violence_ind"] = victims["victim_uid"].isin(
    victim_relationships
    .loc[lambda df: df["value"].isin(REL_DV)]
    ["victim_uid"]
).astype(int)

victims["domestic_violence_ind"].mean()

0.12748602162287873

In [63]:
victims["family_violence_ind"] = victims["victim_uid"].isin(
    victim_relationships
    .loc[lambda df: df["value"].isin(REL_FV)]
    ["victim_uid"]
).astype(int)

victims["family_violence_ind"].mean()

0.1523795113627289

In [64]:
(victims["domestic_violence_ind"] | victims["family_violence_ind"]).mean()

0.23661501216759762

# Standardize data

In [65]:
victims_standardized = (
    victim_top_offenses
    [[
        "victim_uid",
        "offense_uid",
        "UCR OFFENSE CODE"
    ]]
    .merge(
        victims
        [[
            "incident_uid",
            "victim_uid",
            "victim_injury_category",
            "domestic_violence_ind",
            "family_violence_ind",
            "TYPE OF VICTIM",
            "AGE OF VICTIM",
            "SEX OF VICTIM",
            "RACE OF VICTIM",
            "ETHNICITY OF VICTIM",
            "AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 1",
            "AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 2",
        ]],
        how = "left",
        on = [ "victim_uid" ]
    )
    # Exclude non-person victims (e.g., businesses for robberies)
    .loc[lambda x: x["TYPE OF VICTIM"].isin([ "I", "L" ])]
    .assign(
        victim_officer_ind = lambda x: (x["TYPE OF VICTIM"] == "L").astype(int)
    )
    .drop(columns = [ "TYPE OF VICTIM" ])
    .merge(
        offenses
        [[
            "offense_uid",
            "primary_weapon_category",
            "firearm_ind",
            "LOCATION TYPE",
        ]],
        how = "left",
        on = [ "offense_uid" ]
    )
    .assign(
        location_home_ind = lambda df: (df["LOCATION TYPE"] == "20").astype(int),
    )
    .drop(columns = [ "LOCATION TYPE" ])
    .merge(
        incidents
        .merge(
            first_arrest_dates,
            on = [ "incident_submission_uid" ],
            how = "left"
        )
        [[
            "incident_uid",
            "ORIGINATING AGENCY IDENTIFIER ORI",
            "occurred_date",
            "year",
            "month",
            "arrest_ind",
            "arrest_date",
            "exc_clearance_ind",
            "exc_clearance_date",
            "clearance_ind",
            "victim_count",
        ]],
        how = "left",
        on = [ "incident_uid" ]
    )
    .drop(columns = [
        "offense_uid",
    ])
    .rename(columns = {
            "ORIGINATING AGENCY IDENTIFIER ORI": "agency_ori_9",
            "AGE OF VICTIM": "victim_age",
            "SEX OF VICTIM": "victim_sex",
            "RACE OF VICTIM": "victim_race",
            "ETHNICITY OF VICTIM": "victim_ethnicity",
            "AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 1": "circumstance_1",
            "AGGRAVATED ASSAULT HOMICIDE CIRCUMSTANCES 2": "circumstance_2",
            "UCR OFFENSE CODE": "top_offense_code"
    })

)

victims_standardized.head()

,victim_uid,top_offense_code,incident_uid,victim_injury_category,domestic_violence_ind,family_violence_ind,victim_age,victim_sex,victim_race,victim_ethnicity,...,agency_ori_9,occurred_date,year,month,arrest_ind,arrest_date,exc_clearance_ind,exc_clearance_date,clearance_ind,victim_count
0,VA0370000|1996|Z63BRW5HPU72-001,09A,VA0370000|1996|Z63BRW5HPU72,major,0,1,48,F,B,N,...,VA0370000,19960621,1996,6,1,19960621,0,NaN,1,3
1,VA1230000|2015|94-4NLE8O39G-001,09A,VA1230000|2015|94-4NLE8O39G,missing,0,0,21,M,B,N,...,VA1230000,20150411,2015,4,0,NaN,0,NaN,0,3
2,OHCOP0000|2010|8I0VYC5VT2 N-001,09A,OHCOP0000|2010|8I0VYC5VT2 N,missing,0,0,15,M,B,,...,OHCOP0000,20100714,2010,7,0,NaN,0,NaN,0,1
3,CT0004300|2012|2W1J0U73LNK-001,09A,CT0004300|2012|2W1J0U73LNK,major,1,0,28,F,W,N,...,CT0004300,20120519,2012,5,1,20120519,0,NaN,1,2
4,VA1230000|2015|7I-IMIR6ZU72-001,09A,VA1230000|2015|7I-IMIR6ZU72,missing,0,0,24,M,B,N,...,VA1230000,20150517,2015,5,1,20150518,0,NaN,1,4


In [66]:
victims_standardized["top_offense_code"].value_counts().sort_index()

09A      60691
09B       3849
09C       2170
11A     489342
11B     116372
11C      55879
11D     574608
120    1364909
13A    3697940
Name: top_offense_code, dtype: int64

# Save standardized data

In [67]:
victims_standardized.to_csv(
    "../../data/standardized/nibrs-victims.csv",
    index = False
)

**Note**: After this step, delete `../../data/raw/nibrs-temporary-db.sqlite`.

---

---

---